In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd

fake_path = "/content/drive/MyDrive/Fake and Real News Dataset/Fake.csv"
real_path = "/content/drive/MyDrive/Fake and Real News Dataset/True.csv"

In [8]:
df_fake = pd.read_csv(fake_path)
df_real = pd.read_csv(real_path)

In [9]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [10]:
df_real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [11]:
df_fake["Label"] = 1
df_real["Label"] = 0

In [12]:
df_real.head()

,title,text,subject,date,Label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [13]:
df_fake.head()

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [14]:
df = pd.concat([df_real,df_fake], axis=0)

df.head()

,title,text,subject,date,Label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [15]:
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
df.head()

,title,text,subject,date,Label
0,STUNNING DISREGARD FOR LAW: 11 CA Counties Hav...,But..b but it isn t fair to ask for voter ID s...,left-news,"Aug 7, 2017",1
1,Highlights: The Trump presidency on February 3...,(Reuters) - Highlights of the day for U.S. Pre...,politicsNews,"February 3, 2017",0
2,"U.S. Secret Service says 1,100 employees face ...",WASHINGTON (Reuters) - The U.S. Secret Service...,politicsNews,"August 21, 2017",0
3,"Erdogan, Trump agree joint action against Isla...",ANKARA (Reuters) - Turkish President Tayyip Er...,politicsNews,"February 8, 2017",0
4,Sean Hannity Throws Hissy Fit After Real Repo...,"Just like Donald Trump, Sean Hannity has a thi...",News,"August 5, 2016",1


In [17]:
df = df.drop(columns=["title","subject","date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44898 non-null  object
 1   Label   44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 701.7+ KB


In [18]:
print(df.isnull().sum())

text     0
Label    0
dtype: int64


In [19]:
df["text"] = df["text"].str.lower()

In [20]:
import re

def clean_text(text):
  return re.sub(r'[a-zA-Z/s]',"",text)

In [21]:
df["text"] = df["text"].apply(clean_text)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
tfidf_vectorizer = TfidfVectorizer(max_features = 5000)


In [24]:
x = tfidf_vectorizer.fit_transform(df["text"])

In [25]:
y = df["Label"]

In [26]:
df.head()

,text,Label
0,".. . - , ...",1
1,"() - .. ’ : - 2008 , ...",0
2,"() - .. 1,100 - , ’ ...",0
3,"() - .. -, , ...",0
4,", . ...",1


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [29]:
print("Training Data Shape:",X_train.shape,y_train.shape)
print("Testing Data Shape:", X_test.shape,y_test.shape)

Training Data Shape: (35918, 3981) (35918,)
Testing Data Shape: (8980, 3981) (8980,)


In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
model = LogisticRegression()

In [32]:
model.fit(X_train,y_train)

LogisticRegression()

In [33]:
y_pred = model.predict(X_test)

In [34]:
print("Predicted Labels:" ,y_pred[:10])
print("Actual Labels:", y_test[:10].values)

Predicted Labels: [1 1 0 0 1 1 1 1 1 0]
Actual Labels: [1 0 0 0 1 0 1 0 1 0]


In [35]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("\nClassification Report:\n", classification_report(y_test,y_pred))

Accuracy: 0.65

Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.55      0.60      4252
           1       0.65      0.73      0.69      4728

    accuracy                           0.65      8980
   macro avg       0.65      0.64      0.64      8980
weighted avg       0.65      0.65      0.65      8980



In [47]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100)

rf_model.fit(X_train,y_train)

RandomForestClassifier()

In [50]:
y_pred_rf = rf_model.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score, classification_report


accuracy_rf = accuracy_score(y_test,y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.2f}")

print("\nCLassification Report:\n", classification_report(y_test,y_pred_rf))

Random Forest Accuracy: 0.71

CLassification Report:
               precision    recall  f1-score   support

           0       0.74      0.59      0.65      4252
           1       0.69      0.81      0.74      4728

    accuracy                           0.71      8980
   macro avg       0.71      0.70      0.70      8980
weighted avg       0.71      0.71      0.70      8980



In [44]:
from sklearn.model_selection import RandomizedSearchCV


In [55]:
param_dict = {
    "n_estimators":[50,100,200],
    "max_depth": [10,20,30,None],
    "min_samples_split":[2,5,10],
    "min_samples_leaf": [1,2,4],
    "max_features":["sqrt","log2", None]
    }

In [56]:
rf_random = RandomizedSearchCV(
    estimator = rf_model,
    param_distributions = param_dict,
    n_iter=20,
    cv = 5,
    verbose =2,
    n_jobs = 1

)

In [58]:
rf_random.fit(X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END max_depth=30, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   1.3s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   1.4s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   2.2s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   4.1s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   2.3s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=  15.3s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   9.8s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=20,
                   n_jobs=1,
                   param_distributions={'max_depth': [10, 20, 30, None],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   verbose=2)

In [59]:
y_train_pred = rf_random.predict(X_train)

y_test_pred = rf_model.predict(X_test)

In [61]:
train_acc = accuracy_score(y_train,y_train_pred)
print("Training Accuracy:", train_acc)

test_acc = accuracy_score(y_test,y_test_pred)
print("Testing Accuracy:",test_acc)

print("Classification Report (Test Set):")
print(classification_report(y_test,y_test_pred))

Training Accuracy: 0.8341221671585277
Testing Accuracy: 0.7055679287305122
Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.74      0.59      0.65      4252
           1       0.69      0.81      0.74      4728

    accuracy                           0.71      8980
   macro avg       0.71      0.70      0.70      8980
weighted avg       0.71      0.71      0.70      8980



In [62]:
import joblib

joblib.dump(model, "Fake_Real_News_model.pkl")

['Fake_Real_News_model.pkl']

In [63]:
from google.colab import files

files.download("Fake_Real_News_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
!ls

drive  Fake_Real_News_model.pkl  sample_data
